# REQUERIMENTS #

In [ ]:
pip install selenium

In [ ]:
pip install webdriver_manager

In [ ]:
pip install openpyxl

# Variables to define #

## User & Password ## 

## Keyword ##

is the word for searchs, such as "java usd", "python usd", "python jr", "java jr" etc

## Scroll ##

quantity of scrollings web pages, one scrolling aproximate 6 to 9 publications

## Language ##

1 = Spanish users         
2 = English users

In [ ]:
################# Variables to define #################
username_linkedin = ""
password_linkedin = ""
keyword = ""
scrolls = 3
language = 1
#######################################################

In [ ]:
from email import header
from operator import index
from wsgiref import headers
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait 
import pandas as pd
import time
import csv

s=Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(service=s,options=options)
driver.maximize_window()
driver.get("http://linkedin.com")

time_sleeps = 5

time.sleep(time_sleeps)
sesion = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, './/a[@class = "nav__button-secondary"]'))).click()

#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='session_key']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='session_password']")))

publications_list = []
publications_list_aux = []
link_autors_list_aux = []
link_autors_list = []
head_autors_list_aux = []
name_autors_list = []
grade_contact_list = []
job_title_autors_list = []
date_publications_list = []
templist = []
i = 0

scroll_publications_index = './/div[@class = "ph0 pv0 search-results__no-cluster-container mb2"]'
publication_index1 = './/div[@class = "feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative feed-shared-update-v2--e2e artdeco-card ember-view"]'
publication_index2 = './/div[@class = "feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative artdeco-card ember-view"]'
link_autor = '//a[@Class = "app-aware-link feed-shared-actor__container-link relative display-flex flex-grow-1"]'
publications_text = '//span[@class="break-words"]//span'
head_autor = '//div[@Class = "feed-shared-actor__meta relative"]'

if language == 1:
    search_button = "//input[@placeholder='Buscar']"
    publication_button = '//button[normalize-space()="Publicaciones"]'
else:
    search_button = "//input[@placeholder='Search']"
    publication_button = '//button[normalize-space()="Posts"]'


username.clear()
username.send_keys(username_linkedin)
password.clear()
password.send_keys(password_linkedin)

time.sleep(time_sleeps)
#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

print("We are logged in!")

   
#target the search input field
time.sleep(time_sleeps)
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, search_button)))
searchbox.clear()
searchbox.send_keys(keyword)
 
time.sleep(time_sleeps)
searchbox.send_keys(Keys.ENTER)
print("Searching for " + str(keyword))

publicaciones = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, publication_button))).click()
#filtros = sesion = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Iniciar sesión")]'))).click()
print("Filtering in publications")

time.sleep(time_sleeps)
while i < scrolls:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print("Scrolling " + str(i+1) + " of " + str(scrolls))
    time.sleep(time_sleeps)
    i += 1

print("Scrolling done, getting the data")
time.sleep(time_sleeps)
posts_all = driver.find_elements(By.XPATH, scroll_publications_index)

In [ ]:
for element in posts_all:
    publications_list_aux.append(element.find_elements(By.XPATH, publication_index1 + publications_text))
    publications_list_aux.append(element.find_elements(By.XPATH, publication_index2 + publications_text))
    link_autors_list_aux.append(element.find_elements(By.XPATH, publication_index1 + link_autor))
    link_autors_list_aux.append(element.find_elements(By.XPATH, publication_index2 + link_autor))
    head_autors_list_aux.append(element.find_elements(By.XPATH, publication_index1 + head_autor))
    head_autors_list_aux.append(element.find_elements(By.XPATH, publication_index2 + head_autor))
  
    
for posts in publications_list_aux:
    for post in posts:
        publications_list.append(post.text)

i = 0
while i < len(publications_list):
    if i != (len(publications_list) -1):
        if(publications_list[i] == publications_list[i+1]):
            publications_list.pop(i+1)
            
    i += 1  
        
for links in link_autors_list_aux:
    for link in links:
        link_autors_list.append(link.get_attribute("href"))

for heads in head_autors_list_aux:
    for head in heads:
        templist = head.text.split("\n")
        name_autors_list.append(templist[0])
        grade_contact_list.append(templist[1])
        job_title_autors_list.append(templist[2])
        date_publications_list.append(templist[3])

In [ ]:
#setting our dataframe

dict = {'Nombre': name_autors_list,
     'Perfil': link_autors_list,
     'Grado Contacto': grade_contact_list,
     'Puesto': job_title_autors_list,
     'Fecha Publicacion': date_publications_list,
     'Publicacion': publications_list
    }

df = pd.DataFrame.from_dict(dict)
print("Indexing the data")

#saving the data
df.to_excel('LinkedinSearch for ' + keyword + '.xlsx', index = False)
driver.close()
print("All data saved. Your search it's done!")
